In [22]:
import pandas as pd # allows us to work with tabular data
import requests # to send the API requests to Elsevier
import json # to read the JSON data that is returned by the APIs
import pickle # pickle files are a good way to save data for reuse in Python
from datetime import datetime

In [23]:
api_key = 'your elsevier api key'

In [24]:
'''search parameters: add values to the dictionary key:value pairs below
leave any fields that you don't want to use as either 0 or '''

search_string_parameters = {
    'ISSN' : '', # Limit results to results from a journal ISSN (add as a string). Leave = '' if no value.
    'AUTHOR-NAME' : '', # Search the author name field. lastname, firstname. e.g., Noble, Safiya
    'KEY' : '', # Add a keyword to search in the text of the article
    'PUBLISHER' : '', # Add a publisher name. e.g., Springer
    'EXACTSRCTITLE' : 'journal', # Add keywords that appear in the journal, book, or conference title. e.g., Informatics
    'AFFIL' : '', # add keywords that appear in the author's institutional affiliation. e.g., to find the University of Minnesota, search for University and Minnesota
    'TITLE' : 'Digital Twin', # Add keywords that appear in the article or chapter title. Can use AND, OR, and AND NOT. e.g. cat AND dog 
    'OPENACCESS' : '1', # Add 1 to limit your results to open access articles or 0 for articles that are not open access
    'start_year' : 2013, # Limit your search to items published after this year. Leave = 0 if no value. YYYY , e.g., 1995
    'end_year' : 2024 # Limit your search to items published before this year. Leave = 0 if no value. YYYY, e.g., 2020
}
def search_builder(search_string_parameters):
    ''' Concatenates a search string query formatted for the Scopus Search API.
    - search_string_parameters expects a python dictionary with keys aligned to API search fields.
    '''
    search_string = ''

    for k,v in search_string_parameters.items():
        if k == 'start_year' and v:
            if search_string != '':
                search_string += f' AND '
            search_string += f'PUBYEAR > {v}'
        elif k == 'end_year' and v:
            if search_string != '':
                search_string += f' AND '
            search_string += f'PUBYEAR < {v}'
        elif v:
            if search_string != '':
                search_string += f' AND '
            search_string += f'{k}({v})'

    return search_string

def create_url(search_string):
    '''Accepts a formatted search string that will be added to the Scopus Search API URL. 
    Requires a global api_key variable.
    Formats and returns a URL to send to the Scopus Search API.'''
    
    query = f'{search_string}'
    url_template = 'https://api.elsevier.com/content/search/scopus?query={query}&apiKey={api_key}'
    full_url = url_template.format(query=query, api_key=api_key)
    return full_url

def connect_to_endpoint(full_url, params={'cursor': '*'}, next_ = '*'):
    '''Accepts API URL with ISSN, default parameters, and next page cursor;
    Sends request to Scopus API and collects JSON results for each call;
    Returns r.json() for the ['search-results'] key.'''
    
    params['cursor'] = next_
    r = requests.get(full_url, params=params)
    r.raise_for_status()
    return r.json()['search-results'], r.headers

In [25]:
search_string = search_builder(search_string_parameters)
print(search_string)
full_url = create_url(search_string)
r_json, r_headers = connect_to_endpoint(full_url)

EXACTSRCTITLE(journal) AND TITLE(Digital Twin) AND OPENACCESS(1) AND PUBYEAR > 2013 AND PUBYEAR < 2024


In [26]:
print('Title:', r_json['entry'][0]['dc:title'], 
        '\nCreator:', r_json['entry'][0]['dc:creator'], 
        '\nPublication:', r_json['entry'][0]['prism:publicationName'],
        '\nDate:', r_json['entry'][0]['prism:coverDate'],
        '\nISSN:', r_json['entry'][0]['prism:eIssn'], 
        '\nDOI:', r_json['entry'][0]['prism:doi'],
        '\nAbstract', r_json['entry'][0])

Title: Internet of Things and Digital Twin Technology-Based Management System of Medical Equipment 
Creator: Liu W. 
Publication: Global Clinical Engineering Journal 
Date: 2023-12-10 
ISSN: 25782762 
DOI: 10.31354/globalce.v6i1.164 
Abstract {'@_fa': 'true', 'link': [{'@_fa': 'true', '@ref': 'self', '@href': 'https://api.elsevier.com/content/abstract/scopus_id/85179342788'}, {'@_fa': 'true', '@ref': 'author-affiliation', '@href': 'https://api.elsevier.com/content/abstract/scopus_id/85179342788?field=author,affiliation'}, {'@_fa': 'true', '@ref': 'scopus', '@href': 'https://www.scopus.com/inward/record.uri?partnerID=HzOxMe3b&scp=85179342788&origin=inward'}, {'@_fa': 'true', '@ref': 'scopus-citedby', '@href': 'https://www.scopus.com/inward/citedby.uri?partnerID=HzOxMe3b&scp=85179342788&origin=inward'}], 'prism:url': 'https://api.elsevier.com/content/abstract/scopus_id/85179342788', 'dc:identifier': 'SCOPUS_ID:85179342788', 'eid': '2-s2.0-85179342788', 'dc:title': 'Internet of Things and

In [27]:
dois = [entry.get('prism:doi') for entry in r_json.get('entry', [])]
print(dois[0])

10.31354/globalce.v6i1.164


In [28]:
def create_url(doi):
    """Accepts a DOI as a string that will be added to the Abstract Retrieval API URL. 
    Requires a global api_key variable.
    Formats and returns a URL to send to the Abstract Retrieval API.
    """
    doi = f'{doi}'
    url_template = 'https://api.elsevier.com/content/abstract/doi/{doi}?&apiKey={api_key}'
    full_url = url_template.format(doi=doi, api_key=api_key)
    return full_url

def connect_to_endpoint(full_url):
    '''Input full_url from create_url function;
    Send request to Scopus Abstract Retrieval API
    Returns r.json response;'''
    
    r = requests.get(full_url, headers =  {'Accept': 'application/json'})
    r.raise_for_status()
    return r.json(), r.headers

In [29]:
full_url = create_url(dois[0])

In [30]:
r_json, r_headers = connect_to_endpoint(full_url)
r_json = r_json['abstracts-retrieval-response']
print(r_json['coredata']['dc:description'])

Background: In recent years medical technology has progressed with the rapid development of medical services and required optimization of medical equipment. However, a lack of effective management methods has led to the inefficient use of medical equipment. Therefore, an effective medical equipment management mode is urgently needed to address these problems and challenges. Methods: The Internet of Things and digital twin technology are applied to intelligent medical equipment management as the current standard of medical equipment management. Results: The intelligent perception terminal can realize the dynamic acquisition of real data, such as the location, process, and efficient use of medical equipment, and help carry out digital, networked, and intelligent monitoring and analysis. Meanwhile, applications such as dynamic management software, real-time positioning software, and space-environment quality monitoring software are being developed. Conclusion: Automatic, intelligent, and 

In [31]:
from langchain_community.chat_models import ChatOpenAI

In [32]:
chat_model = ChatOpenAI(openai_api_key="your openai api key")

In [33]:
abstract = r_json['coredata']['dc:description']
result = chat_model.predict(abstract + " 이 내용을 250자 내외로 요약해줘")
print(result)

최근 의료기술의 발전으로 의료서비스가 급속히 발전하고 의료장비의 최적화가 필요하게 되었습니다. 그러나 효과적인 관리 방법의 부족으로 의료장비의 비효율적인 사용이 발생하고 있습니다. 따라서 이러한 문제와 도전에 대응하기 위해 효과적인 의료장비 관리 모드가 절실히 필요합니다. 이 연구에서는 사물인터넷과 디지털 트윈 기술을 의료장비 관리의 현재 표준으로 삼아 지능형 의료장비 관리에 적용하였습니다. 연구 결과로는 지능형 인지 단말기를 통해 의료장비의 위치, 공정, 효율적인 사용 등의 실시간 데이터 수집이 가능해지며, 디지털화, 네트워크화, 지능화된 모니터링 및 분석이 가능해집니다. 동시에 동적 관리 소프트웨어, 실시간 위치 추적 소프트웨어 및 공간 환경 품질 모니터링 소프트웨어 등의 응용 프로그램이 개발되고 있습니다. 결론적으로 디지털 트윈을 기반으로 한 의료장비 구성, 운영 및 성능 평가의 자동화, 지능화 및 시각화된 관리는 협업적인 관리 효율과 운영 리소스 지원을 개선할 수 있습니다.


In [34]:
from openai import OpenAI

In [35]:
client = OpenAI(api_key="your openai api key")

response = client.images.generate(
    model="dall-e-3",
    prompt=result,
    size="1024x1024",
    quality="standard",
    n=1,
)

image_url = response.data[0].url
print(image_url)

https://oaidalleapiprodscus.blob.core.windows.net/private/org-Uq2LWMd90SmdTeriwBNEBnCy/user-MeAjquxmeL8IJoFSwXdOdKtU/img-jeFmdbkbYO79T7Kv6UGJccs7.png?st=2024-01-16T03%3A21%3A53Z&se=2024-01-16T05%3A21%3A53Z&sp=r&sv=2021-08-06&sr=b&rscd=inline&rsct=image/png&skoid=6aaadede-4fb3-4698-a8f6-684d7786b067&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2024-01-16T04%3A09%3A30Z&ske=2024-01-17T04%3A09%3A30Z&sks=b&skv=2021-08-06&sig=Be9G14UNvBYsJb4chFZwiIE/7Dxw7nVTvmPM3PvZ7Pk%3D
